In [ ]:
# Recupero probabilidades de cada modelo
library(data.table)

# Rutas
archivoA <- "/content/buckets/b1/exp/e010_03/e010_03_prediccion.txt"
archivoB <- "/content/buckets/b1/exp/e012_03/e012_03_prediccion.txt"

# Leo archivos de predicciones
predA <- fread(archivoA)
predB <- fread(archivoB)

# Renombro columnas
setnames(predA, "prob", "prob_A")
setnames(predB, "prob", "prob_B")

In [ ]:
# Armo dfuture
dfuture <- dataset[foto_mes %in% PARAM$train_final$future]

# Mergeo predicciones al dfuture
dfuture <- merge(
  dfuture,
  predA[, .(numero_de_cliente, foto_mes, prob_A)],
  by = c("numero_de_cliente", "foto_mes"),
  all.x = TRUE,
  sort = FALSE
)

dfuture <- merge(
  dfuture,
  predB[, .(numero_de_cliente, foto_mes, prob_B)],
  by = c("numero_de_cliente", "foto_mes"),
  all.x = TRUE,
  sort = FALSE
)

In [ ]:
# Promedio simple
dfuture[, prob := (prob_A + prob_B) / 2]


In [ ]:
# Me quedo solo con numero_de_cliente y prob
tb_prediccion <- dfuture[
  !is.na(prob),
  .(numero_de_cliente, prob)
]

# Ordenar por probabilidad descendente
setorder(tb_prediccion, -prob)

# Cantidad de envíos
envios <- 11000

# Extraer solo los IDs
ids_envio <- tb_prediccion[1:envios, .(numero_de_cliente)]

# Nombre del archivo
archivo_csv <- paste0("e013_03_", envios, ".csv")

# Guardar 
fwrite(
  ids_envio,
  file = archivo_csv,
  col.names = FALSE,
  sep = ","
)